In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import NASNetLarge

In [2]:
#Load Data
train = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function = tf.keras.applications.nasnet.preprocess_input
).flow_from_directory(
    "FER2013/train/",
    class_mode="categorical",
    color_mode="rgb",
    batch_size=256,
    target_size = (331, 331)
)
test = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function = tf.keras.applications.nasnet.preprocess_input
).flow_from_directory(
    "FER2013/test/",
    class_mode="categorical",
    color_mode="rgb",
    batch_size=256,
    target_size = (331, 331)
)

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [3]:
input_shape = (331, 331, 3)

In [4]:
base = NASNetLarge(
    input_shape=input_shape,
    include_top=False,
    weights="imagenet",
    pooling = 'avg'
)
base.trainable = False

343613440/343610240 [==============================] - 15s 0us/step


In [5]:
model = keras.Sequential(
    [
        base,
#         layers.Flatten(),
#         layers.Dense(256, activation='relu'),
#         layers.Dense(128, activation='relu'),
#         layers.Dense(64, activation='relu'),
        layers.Dense(7, activation = 'softmax')
    ]
)

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
NASNet (Functional)          (None, 4032)              84916818  
_________________________________________________________________
dense (Dense)                (None, 7)                 28231     
Total params: 84,945,049
Trainable params: 28,231
Non-trainable params: 84,916,818
_________________________________________________________________


In [7]:
model.compile(
    optimizer='adam',
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

In [8]:
check = tf.keras.callbacks.ModelCheckpoint(
    "weights",
    monitor="accuracy",
    verbose=0,
    save_freq="epoch",
)

In [9]:
model.fit(
    x = train,
    epochs = 10,
    callbacks = [check],
    use_multiprocessing = False
)

Epoch 1/10
113/113 [==============================] - ETA: 0s - loss: 1.5601 - accuracy: 0.3949 WARNING:tensorflow:From /opt/conda/lib/python3.8/site-packages/tensorflow/python/training/tracking/tracking.py:111: Model.state_updates (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: weights/assets
113/113 [==============================] - 5277s 47s/step - loss: 1.5601 - accuracy: 0.3949
Epoch 2/10
113/113 [==============================] - 5345s 47s/step - loss: 1.4172 - accuracy: 0.4624
Epoch 3/10
113/113 [==============================] - 5223s 46s/step - loss: 1.3665 - accuracy: 0.4834
Epoch 4/10
113/113 [==============================] - 5299s 47s/step - loss: 1.3272 

KeyboardInterrupt: 

In [ ]:
model.evaluate(x=test, use_multiprocessing = False)